# Análisis y calibración de técnicas de aprendizaje máquina

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
plt.style.use('ggplot')
%matplotlib inline


# Conjuntos de datos de prueba en sklearn que utlizaremos
#----------------------------------------------------------

# Los conjuntos artificiales típicos para probar datos
from sklearn.datasets import make_moons           # En forma de medialunas 
from sklearn.datasets import make_circles         # En forma de círculos
from sklearn.datasets import make_classification  # Como separación lineal

# Conjunto de datos de dífgitos escritos a mano, versión reducida
from sklearn.datasets import load_digits


# Los métodos de aprendizaje a utilizar ya provenientes de sklearn
#------------------------------------------------------------------
from sklearn.neighbors import KNeighborsClassifier      # KNN
from sklearn.svm import SVC                             # SVM
from sklearn.tree import DecisionTreeClassifier         # Arbol decisión
from sklearn.ensemble import RandomForestClassifier     # Bósque aleatorios
from sklearn.ensemble import AdaBoostClassifier         # ADA Boost
from sklearn.naive_bayes import GaussianNB              # Naive bayes
from sklearn.linear_model import  LogisticRegression    # Logística con regularización

# Logística con polinomio de orden 2
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis    


# Métodos de preprocesamiento que utilizaremos
#----------------------------------------------

# Para normalizar los datos (media y desviación estandar)
from sklearn.preprocessing import StandardScaler

# Para hacer análisis en componentes principales
from sklearn.decomposition import PCA


# Métodos de validación cruzada de sklearn que utilizaremos
#-----------------------------------------------------------

# Para separar los datos en entrenamiento y validación
from sklearn.model_selection import train_test_split

# Genera conjuntos de datos para validación cruzada
from sklearn.model_selection import ShuffleSplit

# Para encontrar el mejor valor de un parámetro
from sklearn.model_selection import GridSearchCV


# Métodos de curvas de aprendizaje y análisis 
# ---------------------------------------------

# Para hacer curvas de aprendizaje
from sklearn.model_selection import learning_curve

# Para hacer curvas de validación
from sklearn.model_selection import validation_curve


# Establecer un flujo de trabajo de ML
#-------------------------------------
from sklearn.pipeline import Pipeline


## 1. Comparasión de diferentes clasificadores

Muestra para 3 conjuntos de datos artificiales bidimensionales, la forma en que se realiza la clasificación con distintos métodos. Principalmente lo hacemos para poder sacar conclusiones sobre en que situaciones un método puede ser mejor que otros, y que está haciendo internamente.

Codigo obtenido de la documentación de scikit-learn, el cual se puede consultar [aquí](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).

### 1.1. Generando 3 conjuntos de aprendizaje sintéticos
ara probar como funcionan los diferentes tipos de clasificadores, primero vamos a revisar cual es el tipo de partición del espacio que se espera con cada uno de ellos en 3 casos diferentes. Para los tres casos se va a generar 3 conjuntos de datos sintéticos es dos dimensiones (con el fin de graficar las diferencias).

Estos tres conjuntos son de la siguiente forma:

1. El primer conjunto tiene forma de media luna los datos de una clase respecto a la otra.

2. En el segundo conjunto de datos, los datos de las dos clases están en círculos concéntricos.

3. El tercer caso son datos linealmente separables con ruido.


In [ ]:
# Datos en forma de media luna
X1, y1 = make_moons(noise=0.3, random_state=0)

# Datos en forma de círculos
X2, y2 = make_circles(noise=0.2, factor=0.5, random_state=1)

# Datos en forma de regresion lineal
X3, y3 = make_classification(n_features=2, n_redundant=0, n_informative=2,
                             random_state=1, n_clusters_per_class=1)
# Le agregamos ruido para hacerlos interesantes
rng = np.random.RandomState(2)
X3 += 2 * rng.uniform(size=X3.shape)

# Los conjuntos de datos irdenados como una lista de pares ordenados
datasets = [(X1, y1), (X2, y2), (X3, y3)]

# Y los grafiacamos para verlos
figure = plt.figure(figsize=(30, 10))
cm_escala = ListedColormap(['#FF0000', '#0000FF'])

for (i, ds) in enumerate(datasets):

    # Selecciona los valores del conjunto de datos y los escala
    X, y = ds
    X = StandardScaler().fit_transform(X)

    # Grafica
    ax = plt.subplot(1, 3, i+1)
    ax.scatter(X[:, 0], X[:, 1], c=y, s=150, cmap=cm_escala)
    ax.set_xlim(X[:, 0].min() - .5, X[:, 0].max() + .5)
    ax.set_ylim(X[:, 1].min() - .5, X[:, 1].max() + .5)
    ax.set_xticks(())
    ax.set_yticks(())
figure.subplots_adjust(left=.02, right=.98)    
plt.show()


### 1.2. Definiendo una la bateria de clasificadores diferentes

En esta sección se va a generar una batería de diferentes objetos clasificador, cada uno proveniente de una técnica diferente. Todos los vamos a guardar en una lista de objetos tipo clasificador de `sklearn`.

Una ventaja de `sklearn` es que todos los objetos clasificador se pueden ajustar sus parámetros en la inicialización, y todos (sean del tipo que sean) utilizan varios métodos, siempre de la misma manera, en particular: `clf.fit` para el aprendizaje y `cls.predict`para el reconocimiento.

In [ ]:
clasificadores = [
    KNeighborsClassifier(3),                  # 3 vecinos próximos
    SVC(kernel="linear", C=0.025),            # SVC lineal con C = 0.025
    SVC(gamma=2, C=1),                        # SVC gaussiano con gamma = 2 y C = 1
    DecisionTreeClassifier(max_depth=5),      # Árbol de decisión con máxima profundidad de 5
    RandomForestClassifier(max_depth=5,       # Bósque aleatorios con 10 árboles con una característica
                           n_estimators=10, 
                           max_features=1),
    AdaBoostClassifier(),                     # ADA Boost (con árboles de profundidad 1)
    GaussianNB(),                             # Naive bayes con distribución gaussiana
    LogisticRegression(solver='lbfgs'),       # Logística 
    QuadraticDiscriminantAnalysis()           # Logística con términos cuadráticos sin regularización
]

# Solo para fines de graficación
titulos = ["3 vecinos próximos", 
           "SVM lineal", 
           "SVM gaussiano", 
           "Árbol de desición",
           "Boseques aleatórios", 
           "AdaBoost", 
           "Naive Bayes", 
           "Logística",
           "Discriminante cuadrático"]


### 1.3. Generando la clasificación con cada método diferente

Por cada método establecido, y por cada conjunto de datos, vamos a realizar la clasificación con los datos de aprendizaje, y luego vamos a realizar la predicción con un monton de puntos del espacio (en forma de rejilla) con tal de poner de manifiesto cual es el tipo de partición que induce cada uno de los algoritmos propuestos.

Esto se realiza en forma genética, así que es exactamente igual para todos los mñetodos. Por esto se utilizan varios comandos provenientes de `matplotlib`para generar los datos para reconocer en forma de rejilla, y se realizan algunas operaciones no tan comunes para graficar, que se espera no haya problema en entenderlas, al ser solo un problema técnico.

In [ ]:
## Vamos a escoger una escala de colores de alto contraste
cm = plt.cm.RdBu
cm_escala = ListedColormap(['#FF0000', '#0000FF'])

for (cual, ds) in enumerate(datasets):
    
    print('\n' * 3)
    print("*"*30 + "\n")
    print("Base de datos " + str(cual + 1))
    print("*"*30 + "\n")
    figure = plt.figure(figsize=(30, 30))


    # Escalar y selecciona valores de entrenamiento y prueba
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

    # Meshgrid para pintar las regiones
    xx, yy = np.meshgrid(np.arange(X[:, 0].min() - .5, X[:, 0].max() + .5, 0.02),
                         np.arange(X[:, 1].min() - .5, X[:, 1].max() + .5, 0.02))

    # Por cada clasificador
    for (i, (titulo, clf)) in enumerate(zip(titulos, clasificadores)):
        
        # Escoge el subplot
        ax = plt.subplot(3, 3, i + 1)
        
        # El entrenamiento!!!!
        clf.fit(X_train, y_train)
        
        # Encuentra el error de validación
        score = clf.score(X_test, y_test)

        # Clasifica cada punto en el meshgrid
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z = Z.reshape(xx.shape)

        # Asigna un color a cada punto
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Grafica los datos de entrenamiento y prueba
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_escala, s=150)
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_escala, s=150, alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(titulo, size=30)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=30, horizontalalignment='right')

    figure.subplots_adjust(left=.02, right=.98)
    plt.show()



#### Ejercicio 1.

Para cada una de las técnicas, describe en que casos crees que la técnica sería de las mejores técnics a utilizar como método de clasificación (puedes consultar bibbliografía o solo apoyarte en los resultados, pero tiene que ser congruente).

1. **3 vecinos próximos**: Agrega tu comentario aquí.

2. **SVM lineal**: Agrega tu comentario aquí.

3. **SVM gaussiano**: Agrega tu comentario aquí.

4. **Árbol de desición**: Agrega tu comentario aquí.

5. **Boseques aleatórios**: Agrega tu comentario aquí.

6. **AdaBoost**: Agrega tu comentario aquí.
 
7. **Naive Bayes**: Agrega tu comentario aquí. 

8. **Discriminante lineal**: Agrega tu comentario aquí.

9. **Discriminante cuadrático**: Agrega tu comentario aquí.

## 2. Curvas de aprendizaje

En esta sección vamos a ver como se pueden aplicar tanto las curvas de aprendizaje (para ver si un método es suficiente (o no), así como las curvas de validación (o calibración) para ver si los parámetros que estámos usando provocan sobreaprendizaje o subaprendizaje (o están bien, por supuesto). 

Para esto vamos a utilizar una famosisima base de datos, la de los dígitos escritos a mano.

In [ ]:
digits = load_digits()
X, y = digits.data, digits.target

figure = plt.figure(figsize=(8, 8))

for index, (image, label) in enumerate(zip(digits.images[:16], digits.target[:16])):
    plt.subplot(4, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Este es un %i' % label)


### 2.1 Curvas de aprendizaje

Para hacer las curvas de aprendizaje vamos a hacer una función envolvente (wrap function) de manera que sea más fácil aplicarla a diferentes ejemplos. Esta función envolvente es propuesta dentro de la documentación de `sklearn`.

In [ ]:
def plot_curva_aprendizaje(estimator, title, X, y, ylim=None, cv=None,
                           n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):
    """
    Genera una curva de aprendizaje

    estimator : Objeto clasificador con métodos `fit` y `predict`

    title : Titulo de la figura.

    X : `ndarray` con shape (n_nuestras, n_atributos)

    y : `ndarray` con shape (n_muestras) 

    ylim : tupla (ymin, ymax), opcional. Máximos y mínimos en la gráfica.

    cv : int, cross-validation generator, opcional. Número de folders en K-fold-cross-validation

    n_jobs : int, opcional. Número de tareas en paralelo.
    
    train_sizes : ndarray. Valores a los cuales se hace un punto para gener la curva de aprendizaje
    """

    # EL marco de la gráfica
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel(u"Ejemplos de entrenamiento")
    plt.ylabel(u"Error de predicción")
    
    ####################################################################
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    ####################################################################
    
    train_mean = np.mean(1 - train_scores, axis=1)
    train_std = np.std(1 - train_scores, axis=1)
    test_mean = np.mean(1 - test_scores, axis=1)
    test_std = np.std(1 - test_scores, axis=1)

    plt.fill_between(train_sizes, train_mean - train_std,
                     train_mean + train_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_mean - test_std,
                     test_mean + test_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_mean, 'o-', color="r",
             label=u"Entrenamiento")
    plt.plot(train_sizes, test_mean, 'o-', color="g",
             label=u"Validación")

    plt.legend(loc="best")
    return plt


### 2.2. Analizando el clasificador naive bayes

Ahora vamos a utilizar el clasificador *Naive Bayes* y vamos a ver que problemas podría presentar en la clasificación. En el caso del Naive Bayes, al ser un modelo generativo, el aprendizaje es muy rápido, por lo que vamos a aplicar el método unas 100 veces, pero sin validación cruzada, ya que en ese caso sería muy lento. De esta manera, vamos a tener una estimación correcta no solo de la curva de aprendizaje promedio, si no de su envolvente a una desviación estandar hacia arriba y hacia abajo.

In [ ]:
# Validación cruzada con 100 folders (20 por ciento de datos de validación)
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

# Clasificador tipo Naive bayes gaussiano
clf = GaussianNB()

#Curva de aprendizaje
plt.rcParams['figure.figsize'] = (15, 5)
plot_curva_aprendizaje(clf, "Curva de aprendizaje (Naive Bayes)", 
                       X, y, ylim=(0, 0.3), cv=cv, n_jobs=4)

#### Ejercicio 2.

Responde a las siguiente preguntas:

1. ¿Hay alto sezgo, alta varianza, o el resultado es correcto?
2. ¿Hay sobre aprendizaje, sub aprendizaje o es correcto?
3. ¿Se necesitarían más datos?
4. Indica 3 posibles acciones a considerar en este caso (de ser necesario).


## 2.3. Analizando una máquina de vector de soporte con kernel gaussiano

En este caso, vamos a analizar un clasificador cuyo aprendizaje es más complejo (y por lo tanto toma más tiempo). Por esta razón vamos a limitarnos a una validación cruzada de 10 folders.

In [ ]:
# Validación cruzada de 10 folders (20 por ciento de datos de validación)
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

# Clasificador SVM con kernel gaussiano y gamma=0.001
clf = SVC(gamma=0.001)


#Curva de aprendizaje
plt.rcParams['figure.figsize'] = (15, 5)
plot_curva_aprendizaje(clf, "Curva de aprendizaje (SVM, kernel RBF, $\gamma=0.001$)", 
                       X, y, ylim=(0, 0.1), cv=cv, n_jobs=4)


#### Ejercicio 3.

Responde a las siguiente preguntas:

1. ¿Hay alto bias, alta varianza, o el resultado es correcto?
2. ¿Hay sobre aprendizaje, sub aprendizaje o es correcto?
3. ¿Se necesitarían más datos?
4. Indica 3 posibles acciones a considerar en este caso (de ser necesario).


## 2.4. Curva de validación para el parámetro $\gamma$

Vamos ahora a graficar la curva de validación para diferentes parámetros de $\gamma$ sobre el clasificador por máquinas de vactores de soporte con kernel gaussiano. Dado que los cambios que se pueden percibir son muy pequeños, a menos que el parámetro de varía en forma logarítmica (esto es, en relación a sus ordenes de magnitud), pues la curva de validación la vamos a realizar utilizando una escala logarítmica.

In [ ]:
# Los valores que le vamos a dar a \gamma
param_range = np.logspace(-6, -1, 5)

# La curva de validación con 10 fold-cross-validation
train_scores, test_scores = validation_curve(SVC(), X, y, 
                                             param_name="gamma", 
                                             param_range=param_range,
                                             cv=10, 
                                             scoring="accuracy", 
                                             n_jobs=1)

train_mean = np.mean(1 - train_scores, axis=1)
train_std = np.std(1 - train_scores, axis=1)

test_mean = np.mean(1 - test_scores, axis=1)
test_std = np.std(1 - test_scores, axis=1)


plt.rcParams['figure.figsize'] = (15, 5)

# Plot de los resultados de entrenamiento
plt.semilogx(param_range, train_mean, label="Entrenamiento", color="r")
plt.fill_between(param_range, 
                 train_mean - train_std,
                 train_mean + train_std, 
                 alpha=0.2, color="r")

# Plot de los resultados de validación
plt.semilogx(param_range, test_mean, label=u"Validación cruzada", color="g")
plt.fill_between(param_range, 
                 test_mean - test_std,
                 test_mean + test_std, 
                 alpha=0.2, color="g")

plt.title(u"Curva de validación de SVM con kernel gaussiano al variar $\gamma$")
plt.xlabel(u"Parámetro $\gamma$")
plt.ylabel(u"Error de clasificación")
plt.legend(loc="best")
plt.ylim(0.0, 1.1)



#### Ejercicio 4.

Responde a las siguiente preguntas:

1. ¿En que valor de $\gamma$ hay claramente sobreaprendizaje?
2. ¿En que valor de $\gamma$ hay claramente subaprendizaje?
3. ¿Cual sería a t consideración el mejor valor de $\gamma$?


## 3. Automatización de tareas

Hacer las tareas repetitivas, tales como ajustar parámetros, o utilizar una serie de pasos en un proceso de reconocimiento de partones, son el tipo de problemas que son claramente automatizables. En esta sección vamos a ver como automatizar dos tareas: el uso de un método de preprocesamiento de señales conectado a un mñetodo de clasificación utilizando un `pipeline`; y el proceso de optimización de los valores de los parámetros utilizando curvas de validación (calibración). 

Recuerda que estos son solo dos ejemplos de formas de automatización y que existen otros más ya contemplados dentro de la librería `sklearn`.

### 3.1. Automatización de procedimientos que se realizan en serie

Vamos a asumir que queremos realizar el reconocimiento de digitos escritos a mano (el mísmo conjunto de datos original), pero ahora utilizando otro enfoque. Vamos a preprocesar los datos utilizando análisis en componentes principales (PCA), y luego aplicando un clasificador por regresión logística con regularización. 

Es importante de ver que en el aprendizaje hay que ajustar la matriz de transformación y la normalización de datos en el PCA, para luego utilizar esos resultados para el ajuste de los pesos del regresor logístico. De la misma manera, para reconocer nuevos datos, es necesario normalizarlos y rotarlos de acuerdo al algoritmo de PCA, para que su resultado sea utilizado dentro de la regresión logística. Así, las operacions (tanto de reconocimiento como aprendizaje) se hacen en serie, o en `pipeline` para utilizar un término muy común para quienes utilizan la linea de comandos en UNIX.

Vamos entonces a definir un *pipeline* con estas dos unidades de procesamiento


In [ ]:
# El tratamiento de los datos por PCA 
# con la asignación por default de sus parámetros
pca = PCA()

# El clasificador, por regresión logística
# con la asignación por default de sus parámetros
logistic = LogisticRegression(solver='liblinear', multi_class='ovr')

# Siempre que se haga entrenamiento o predicción, hay que
# aplicar pca primero y logistic después
pipe = Pipeline(
    steps=[
        ('acp', pca), 
        ('logistica', logistic)
    ]
)

# Ahora en pipe tenemos un clasificador, de tal forma que 
# pipe.fit(X, y) ajusta ambos métodos, y
# pipe.predict(X) realiza la prediccion (entre otros operadores genéricos de un clasificador)


### 3.2. Optimizador de parámetros por curvas de validación

Como es posible observar, no es necesario visualizar las curvas de calibración, ya que el mejor valor para una variable es cuando el error de validación es el menor posible. Así que es claro que la tarea de encontra el mejor valor posible para uno o varios parámetros, es probar sobre un conjunto de valores, y seleccionar los valores de los parámetros que ofrezcan el menor error de validación posible.

Al ser esta una tarea fácilmente automatizable, es de esperar que exista ya algo así dentro del modulo de `sklear`. Y efectivamente, existe, para un conjunto finito y predefinido de valores en los cuales buscar.

In [ ]:
# Se escogen dos parámetros a ajustas: 
#       a) El número de componentes principales del PCA
#       b) El valor de C (regularización) para la regresión logística

# EL número de componentes principales podrá ser 20, 40 o todos
n_componentes = [20, 40, 64]

# El parámetro C varia como 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000
Cs = np.logspace(-4, 4, 8)

# Genera un clasificador con optimizador de parámetros en el aprendizaje
#        a) Utiliza el clasificador definido en pipe
#        b) Los parámetros a ajustar se ponen en un diccionario {var1: valores_var1, var2:valores_var2}
#        c) pca__n_components es equivalente a pipe.pca.n_components (para poderlo poner en strings)
#        d) logistic__C es equivalente a la variable pipe.logistic.C
# 
# Al ser el clasificador un pipeline, los parámetros se encuentran en esa forma de la estructura.

clf = GridSearchCV(
    pipe,
    dict(
        acp__n_components= n_componentes,
        logistica__C= Cs
    ),
    cv=3
)

# Ahora clf es un objeto clasificador, que si se ejecuta clf.fit(X,y)
# ajusta los objetos pca y logistic y ajusta los parámetros pca.n_components y logistic.C de
# acuerdo al conjunto de posibles parámetros que le introducimos. El reconocimiento y el resto de
# las funciones son similares a las que tenía pipe



### 3.3 Aplicando el aprendizaje

In [ ]:
clf.fit(X, y)

print("El mejor valor de regularización es: {}".format(clf.best_estimator_.named_steps['logistica'].C))
print("El número óptimo de componentes principales es: {}".format(clf.best_estimator_.named_steps['acp'].n_components))

#### Ejercicio 5.

Prueba de realizar la clasificación de los digitos utilizando un clasificador por máquina de vector de soporte con kernel lineal combinado con análisis en componentes principales.

1.  Encuentra los mejores valores para al menos dos parámetros diferentes.

2. Grafica la curva de aprendizaje y escribe las posibles ventajas y desventajas del método en relación a los métodos de Naive Bayes y de SVM con kernel gaussiano vistos anteriormente

